In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import sqlite3

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import string
import re
import os
import pickle
from tqdm import tqdm
from bs4 import BeautifulSoup

import seaborn as sns

from sklearn import metrics
from sklearn.metrics import confusion_matrix, auc, roc_curve

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from gensim.models import Word2Vec, KeyedVectors 

## [1]. Reading Data

In [2]:
!ls


ai-14-tf-idf-code-sample-ref.ipynb
ai-14-tf-idf-code-sample-self.ipynb
database.sqlite


In [3]:
conn = sqlite3.connect('../data/data_01_amazon_fine_food_review/database.sqlite')
print(conn)

In [4]:
df = pd.read_sql_query("SELECT * FROM Reviews LIMIT 5427", conn)

In [5]:
print(df.head(n = 3))

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1      Not as Advertised  Product arrived labeled as Jumbo Salted Peanut...  
2  "Delight" says it all  This is a confection that has been around a fe...  


### [1.] Available columns in DataFrame

In [6]:
print(df.columns)

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


### [1.] Columns and their DataType

In [7]:
print(df.dtypes)

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object


> For Amazon fine food review database, It has one column `score`
<br>


### [1.]. Filter out reviews having score equal to 3 as  these are consider as neutral

In [8]:
df = df.loc[(df['Score'] < 3) | (df['Score'] > 3)]
print(df.head(n = 3))

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1      Not as Advertised  Product arrived labeled as Jumbo Salted Peanut...  
2  "Delight" says it all  This is a confection that has been around a fe...  


### [1.]. Update existing column score based on belown condition
- Reviews corresponding to score :
  - 4 and 5, would be consider as positive review (1)
  - 1 and 2, would be consider as negative review (0)


In [9]:
df['Score'] = df['Score'].map(lambda score: 1 if score > 3 else 0)
print(df.head(n = 3))

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      1  1303862400   
1                     0                       0      0  1346976000   
2                     1                       1      1  1219017600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1      Not as Advertised  Product arrived labeled as Jumbo Salted Peanut...  
2  "Delight" says it all  This is a confection that has been around a fe...  


In [10]:
# DataFrame shape
print("DataFrame shape : {}".format(df.shape))

DataFrame shape : (5000, 10)


In [11]:
display = pd.read_sql_query("SELECT UserId, ProductId, ProfileName, Score, Time, Text, COUNT(*) AS record FROM Reviews GROUP BY UserId HAVING record > 1", conn)

print("DataFrame shape : {}".format(display.shape))
print(display.head(n = 5))

DataFrame shape : (80668, 7)
               UserId   ProductId             ProfileName  Score        Time  \
0  #oc-R115TNMSPFT9I7  B007Y59HVM                 Breyton      2  1331510400   
1  #oc-R11D9D7SHXIJB9  B005HG9ET0  Louis E. Emory "hoppy"      5  1342396800   
2  #oc-R11DNU2NBKQ23Z  B007Y59HVM        Kim Cieszykowski      1  1348531200   
3  #oc-R11O5J5ZVQE25C  B005HG9ET0           Penguin Chick      5  1346889600   
4  #oc-R12KPBODL2B5ZD  B007OSBE1U   Christopher P. Presta      1  1348617600   

                                                Text  record  
0  Overall its just OK when considering the price...       2  
1  My wife has recurring extreme muscle spasms, u...       3  
2  This coffee is horrible and unfortunately not ...       2  
3  This will be the bottle that you grab from the...       3  
4  I didnt like this coffee. Instead of telling y...       2  


#### [1..1]. Get one user record

In [12]:
print(display[display['UserId'] == '#oc-R12KPBODL2B5ZD'])

               UserId   ProductId            ProfileName  Score        Time  \
4  #oc-R12KPBODL2B5ZD  B007OSBE1U  Christopher P. Presta      1  1348617600   

                                                Text  record  
4  I didnt like this coffee. Instead of telling y...       2  


In [13]:
record_count = display['record'].sum()
print("Record count : {}".format(record_count))

Record count : 393063


## [2]. Exploratory Data Analysis

### [2.]. Data Cleaning: Deduplication

#### [2.]. Sort DataFrame based on ProductId

In [14]:
dup_records = df[df.duplicated(subset = ['UserId', 'ProfileName', 'Time', 'Text'])]
print("Duplicate records DataFrame shape: {}".format(dup_records.shape))
print(dup_records.head(n = 3))

Duplicate records DataFrame shape: (14, 10)
        Id   ProductId          UserId   ProfileName  HelpfulnessNumerator  \
29      30  B0001PB9FY  A3HDKO7OW0QNK4  Canadian Fan                     1   
574    575  B000G6RYNE  A3PJZ8TU8FDQ1K  Jared Castle                     2   
2309  2310  B0001VWE0M   AQM74O8Z4FMS0      Sunshine                     0   

      HelpfulnessDenominator  Score        Time  \
29                         1      1  1107820800   
574                        2      1  1231718400   
2309                       0      0  1127606400   

                                          Summary  \
29                The Best Hot Sauce in the World   
574   One bite and you'll become a "chippoisseur"   
2309                               Below standard   

                                                   Text  
29    I don't know if it's the cactus or the tequila...  
574   I'm addicted to salty and tangy flavors, so wh...  
2309  Too much of the white pith on this orange pee

In [15]:
sorted_df = df.sort_values("ProductId", axis = 0, ascending  = True, inplace = False, kind = 'quicksort', na_position = 'last')
print(sorted_df.head(n = 10))
print(sorted_df.shape)

        Id   ProductId          UserId                           ProfileName  \
2773  2774  B00002NCJC  A196AJHU9EASJN                          Alex Chaffee   
2774  2775  B00002NCJC  A13RRPGE79XFFH                              reader48   
1243  1244  B00002Z754  A3B8RCEI0FXFI6                             B G Chase   
1244  1245  B00002Z754  A29Z5PI9BW2PU3                                Robbie   
3203  3204  B000084DVR  A1UGDJP1ZJWVPF          T. Moore "thoughtful reader"   
3202  3203  B000084DVR  A3DKGXWUEP1AI2           Glenna E. Bauer "Puppy Mum"   
1160  1161  B000084E1U  A3DH85EYHW4AQH                          Eric Hochman   
2379  2380  B0000CGFV4  A3LSUKN4IFS6VD                         Internet Diva   
5096  5097  B0000D94SZ  A2R91PG1XPNO0B  Kotton Kandee "not so secretshopper"   
2255  2256  B0000DC5IY    AGYZZ3QXV9S8      Dr. Glenn B. Gero "NJnaturaldoc"   

      HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
2773                     0                    

In [29]:
final = sorted_df.drop_duplicates(subset = ['UserId', 'ProfileName', 'Time', 'Text'])
print(final.shape)
print(final.head())

(4986, 10)
        Id   ProductId          UserId                   ProfileName  \
2773  2774  B00002NCJC  A196AJHU9EASJN                  Alex Chaffee   
2774  2775  B00002NCJC  A13RRPGE79XFFH                      reader48   
1243  1244  B00002Z754  A3B8RCEI0FXFI6                     B G Chase   
1244  1245  B00002Z754  A29Z5PI9BW2PU3                        Robbie   
3203  3204  B000084DVR  A1UGDJP1ZJWVPF  T. Moore "thoughtful reader"   

      HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
2773                     0                       0      1  1282953600   
2774                     0                       0      1  1281052800   
1243                    10                      10      1   962236800   
1244                     7                       7      1   961718400   
3203                     1                       1      1  1177977600   

                             Summary  \
2773                   thirty bucks?   
2774                    Flies Begone 

In [30]:
#Checking how much % of data still remains
rem_data = np.divide(final['Id'].size, df['Id'].size)
print(rem_data)

0.9972


### [2.]. Data Cleaning: invalid data check

In [31]:
data_sanity_i = final[final['HelpfulnessNumerator'] <= final['HelpfulnessDenominator']]  
print("Invalid DataFrame shape: {}".format(data_sanity_i.shape))
print(data_sanity_i.head())

Invalid DataFrame shape: (4986, 10)
        Id   ProductId          UserId                   ProfileName  \
2773  2774  B00002NCJC  A196AJHU9EASJN                  Alex Chaffee   
2774  2775  B00002NCJC  A13RRPGE79XFFH                      reader48   
1243  1244  B00002Z754  A3B8RCEI0FXFI6                     B G Chase   
1244  1245  B00002Z754  A29Z5PI9BW2PU3                        Robbie   
3203  3204  B000084DVR  A1UGDJP1ZJWVPF  T. Moore "thoughtful reader"   

      HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
2773                     0                       0      1  1282953600   
2774                     0                       0      1  1281052800   
1243                    10                      10      1   962236800   
1244                     7                       7      1   961718400   
3203                     1                       1      1  1177977600   

                             Summary  \
2773                   thirty bucks?   
2774        

#### [2.]. Data Cleaning: After data sanity group data based on negative and positive review

In [33]:
aggregated_scores = data_sanity_i['Score'].value_counts()
print(aggregated_scores)

1    4178
0     808
Name: Score, dtype: int64


## [3]. Text processing

#### [3.]. Get Text insight

In [63]:
# Print randomly data
text_list = data_sanity_i['Text'].values
print("Series.values data type : {}".format(type(text_list)))
print("Columns Text lenght : {}".format(text_list.size))

Series.values data type : <class 'numpy.ndarray'>
Columns Text lenght : 4986


In [64]:
#text_list.where(lambda review_text: )

In [65]:
# Display few Text records randomly
text_0 = text_list[0]
print(text_0)
print("===="*20)

text_1000 = text_list[1000]
print(text_1000)
print("===="*20)

text_2500 = text_list[2500]
print(text_2500)
print("===="*20)

text_4900 = text_list[4900]
print(text_4900)
print("===="*20)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
I love coffee and at night really miss out if I don't have

#### [3.]. Remove URL's from text

In [66]:
print(text_0)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [67]:
#re.findall('https\S+|http\S+', text_0)
print(re.sub('https\S+|http\S+', "", text_0))

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [71]:
data_sanity_i['Text'] = data_sanity_i['Text'].str.replace('https\S+|http\S+', "")
print(data_sanity_i['Text'][0])                                                          

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
"""
text_0
text_1000
text_2500
text_4900
"""